In [1]:
import tarfile
import os
import nltk
import numpy as np
import numpy.matlib
import scipy
import sklearn 
import sklearn.decomposition
import matplotlib.pyplot as plt
import sklearn.manifold
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
nltk.download('wordnet') 
nltk.download('stopwords')
nltk.download('omw-1.4')
lemmatizer = nltk.stem.WordNetLemmatizer() 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vera\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Vera\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vera\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vera\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Vera\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [71]:
topics = ['rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey']
newsgroup_topics_documents = []
name_docs = ['101551', '103118', '98657', '52550']
id_docs = []
closest_names = {}
i = 0

for topic in topics:
    path_to_topic = 'C:/disk_D/1_tampere_uni/text data analysis/hw/6/20_newsgroups/' + topic
    topic_documents_names = os.listdir(path_to_topic)
    topic_documents = []
    topic_found = True
    for topic_document_name in topic_documents_names:
        if str(topic_document_name) in name_docs and topic_found:
            id_docs.append(i)
            topic_found = False
        document_file = open(path_to_topic + '/' + topic_document_name, 'r')
        document = document_file.read()
        document_file.close()
        newsgroup_topics_documents.append(document)
        if i in [1561, 795, 1854, 2907]:
            closest_names[i] = topic_document_name
            #print(f'{i} name is: {topic_document_name}')
        i += 1

In [40]:
excludedlinemarkers=['Xref:','Path:','From:','Newsgroups:','Subject:','Summary:', \
                     'Keywords:','Message-ID:','Date:','Expires:','Followup-To:','Distribution:', \
                     'Organization:','Approved:','Supersedes:','Lines:','NNTP-Posting-Host:', \
                     'References:','Sender:','In-Reply-To:','Article-I.D.:','Reply-To:', \
                     'Nntp-Posting-Host:'] 

for d_document in range(len(newsgroup_topics_documents)):
    templines = newsgroup_topics_documents[d_document].splitlines()    
    remaininglines = []    
    for l in range(len(templines)):        
        line_should_be_excluded = 0        
        for m in range(len(excludedlinemarkers)):            
            if len(templines[l]) >= len(excludedlinemarkers[m]):                
                if excludedlinemarkers[m] == \
                                templines[l][0:len(excludedlinemarkers[m])]:                    
                    line_should_be_excluded=1                    
                    break        
        if line_should_be_excluded == 0:           
            remaininglines.append(templines[l])    
    newsgroup_topics_documents[d_document] = '\n'.join(remaininglines)

In [41]:
def tokenize_text(text):
    
    temp_tokenizedtext = nltk.word_tokenize(text)    
    mycrawled_nltktext = nltk.Text(temp_tokenizedtext)    
    
    return mycrawled_nltktext


def lower_case_text(text):
    mycrawled_lowercasetext = [] 

    for k in range(len(text)):        
        lowercaseword = text[k].lower()        
        mycrawled_lowercasetext.append(lowercaseword)    
        
    return mycrawled_lowercasetext

def tagtowordnet(postag):   
    wordnettag = -1   
    if postag[0] == 'N':        
        wordnettag = 'n'   
    elif postag[0] == 'V':        
        wordnettag = 'v'   
    elif postag[0] == 'J':        
        wordnettag = 'a'    
    elif postag[0] == 'R':        
        wordnettag = 'r'    
    return(wordnettag)

def lemmatizetext(nltktexttolemmatize):    
    # Tag the text with POS tags    
    taggedtext = nltk.pos_tag(nltktexttolemmatize)   
    # Lemmatize each word text    
    lemmatizedtext = []    
    for l in range(len(taggedtext)):       
        # Lemmatize a word using the WordNet converted POS tag       
        wordtolemmatize = taggedtext[l][0]        
        wordnettag = tagtowordnet(taggedtext[l][1])        
        if wordnettag != -1:            
            lemmatizedword = lemmatizer.lemmatize(wordtolemmatize,wordnettag)        
        else:            
            lemmatizedword=wordtolemmatize       
            # Store the lemmatized word        
        lemmatizedtext.append(lemmatizedword)
        
    return(lemmatizedtext) 


In [42]:
newsgroup_texts = []
for document in newsgroup_topics_documents:
    text = " ".join(document.split())
    newsgroup_texts.append(text)
    
newsgroups_tokenized = []
for document in newsgroup_texts:
    document = tokenize_text(document)
    newsgroups_tokenized.append(document)
    

newsgroups_lower = []
for document in newsgroups_tokenized:
    document = lower_case_text(document)
    newsgroups_lower.append(document)
    

newsgroups_texts_lemmatized = []
for document in newsgroups_lower:
    document = lemmatizetext(document)
    #document = nltk.Text(document)
    newsgroups_texts_lemmatized.append(document)

In [43]:
import gensim 
# We need to create a tagged version of each document 

gensim_tagged_docs=[] 

for k in range(len(newsgroups_texts_lemmatized)):    
    doctag='doc'+str(k)    
    tagged_document= \
    gensim.models.doc2vec.TaggedDocument( \
                                         newsgroups_texts_lemmatized[k],[doctag])    
    gensim_tagged_docs.append(tagged_document)
    
    # Create a dictionary from the documents 
gensim_dictionary = gensim.corpora.Dictionary(newsgroups_texts_lemmatized)

In [44]:
doc2vecmodel = gensim.models.doc2vec.Doc2Vec(gensim_tagged_docs, \
                                             vector_size=10, window=5, min_count=1, \
                                             workers=4, dm_concat=0)


In [52]:
docs_to_find = doc2vecmodel[id_docs]
closest_to_docs_value = []
closest_to_docs_id = []

In [63]:
for doc_to_find in docs_to_find:
    value_closest = 10e6
    number_of_closest = 0
    for i in range(len(newsgroups_texts_lemmatized)):
        doc_vec = doc2vecmodel[i]
        if (doc_vec==doc_to_find).all():
            continue
        dist = np.linalg.norm(doc_to_find - doc_vec)
        if dist < value_closest:
            value_closest = dist
            number_of_closest = i
    closest_to_docs_value.append(value_closest)
    closest_to_docs_id.append(number_of_closest)

In [73]:
closest_to_docs_id

[1561, 795, 1854, 2907]

In [72]:
closest_names

{795: '103446', 1561: '104711', 1854: '105077', 2907: '105065'}

Each closest document is from another newsgroup topic.

For the first document (101551, from rec.autos) the closest document is 104711 from rec.motorcycles newsgroup topic.
These documents don't look close to each other by the sense of the content. Because 101511 is about the comparison of car prices and 104711 is just so short and looks like a conspiracy theory.

For the document 103118, from rec.motorcycles the closest document is 103446 from rec.autos. 
In general these documents have different main thought but they represent comparison in some sense.

For the document 98657, from rec.sport.baseball the closest document is 105077 from rec.motorcycles.
I can't find the connection in sense between these documents.

For the document 52550, from rec.sport.hockey the closest document is 105065 from rec.sport.baseball.
In closest document (105065) I found an one mention of word "hockey" which correspond to the topic of the document 52550.

Therefore, I have noticed that closest document to the original have less number of words. So may be it would be meaningful to make some limitations about the number of words in the found closest document.